# Lab 04-3: Support Vector Machine
## Exercise: Predicting Iris Species

### Prepare IRIS Dataset

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris

iris = load_iris()

# iris.data contains four column
#   sepal length (cm) / sepal width (cm) / petal length (cm) / petal width (cm)
# iris.target contains one column
#   species of (0,1,2) = (setosa, versicolor, virginica)
iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
iris_tf = pd.DataFrame(data= iris.target, columns= ['species'])

# remove virginica data
iris_df = iris_df.drop(labels=range(100,150), axis=0)
iris_tf = iris_tf.drop(labels=range(100,150), axis=0)

# species are 0 for setosa, 1 for versicolor, and 2 for virginica
# let's find versicolor; this time, set versicolor to 1 and set setosa to -1
def converter(species):
    return 1 if species == 1 else -1

# change species numbers to binary values (1, -1)
iris_tf['species'] = iris_tf['species'].apply(converter)

# Chnage dataset from pandas to numpy
vX = iris_df.to_numpy()
vY = iris_tf['species'].to_numpy()

### Presenting Dataset Samples

In [2]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,100.000000,100.000000,100.000000,100.000000
mean,5.471000,3.099000,2.861000,0.786000
std,0.641698,0.478739,1.449549,0.565153
min,4.300000,2.000000,1.000000,0.100000
25%,5.000000,2.800000,1.500000,0.200000
50%,5.400000,3.050000,2.450000,0.800000
75%,5.900000,3.400000,4.325000,1.300000
max,7.000000,4.400000,5.100000,1.800000


In [3]:
print(vY)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1]


Splitting Data for Training and Testing, and then Normalizing Input Vectors

In [4]:
# We can use train_test_split from sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Splitting dataframe into train & test
X_train, X_test, y_train, y_test = train_test_split(vX, vY, test_size= 0.20, random_state= 101)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Support Vector Machine

$$H_0: Wx^{(i)} + b = 0, \qquad
J = \lambda {\lVert W \rVert}^2 + \sum_{i=1}^{n} \max{\{0, 1-y^{(i)}(Wx^{(i)}+b)\}} $$

If $x^{(i)}$ is inside margin,
$$ {\partial J \over \partial W} = 2\lambda W - y^{(i)}x^{(i)}, \qquad
{\partial J \over \partial b} = - y^{(i)}$$



Training Model with Support Vector Machine

In [5]:
class mySupportVectorMachine:
    def __init__(self):
        # for SVM, use a weight vector not indivisual weights
        self.wegt = np.zeros(4)
        self.bias = 0.0

In [8]:
# weight for 4 input variables: sepal length, sepal width, petal length, petal width
m = mySupportVectorMachine()

# define learning rate, regularization rate & number of epochs
alpha = 0.001
lmbda = 0.1       # lambda is a python keyword
n_epochs = 50

prev_sqr = 0.0 

for epoch in range(n_epochs):
    ### START CODE HERE ###

    y_lin = np.dot(X_train, m.wegt) + m.bias     # Linear Prediction
    y_hat = y_train * y_lin    # Check the Prediction
    for i, val in enumerate(y_hat):
        if val>=1:
            m.wegt = m.wegt - alpha * (2 * lmbda * m.wegt)    # Update Weights
        else:
            m.wegt = m.wegt - alpha * (2 * lmbda * m.wegt - y_train[i] * X_train[i])    # Update Weights
            m.bias = m.bias - alpha * (-y_train[i])    # Update Bias

    ### END CODE HERE ###

    # reduce lambda for smooth convergence
    sqr_wegt = np.sum(m.wegt ** 2)
    if sqr_wegt > prev_sqr:
        lmbda = lmbda / 2
    prev_sqr = sqr_wegt

    # Print Prediction Cost
    if((epoch+1)%10==0):
        ### START CODE HERE ###

        y_margin = np.where(y_hat >= 1, 0, 1 - y_hat)    # Check if input is IN margin (1- ...)
        cost_J = lmbda * np.linalg.norm(m.wegt, ord=2) + np.sum(y_margin)      # Calculate Cost Value

        ### END CODE HERE ###
        print('Epoch: %5d,  cost: %10.8f' % (epoch+1, cost_J))


Epoch:    10,  cost: 2.25148088
Epoch:    20,  cost: 0.77377965
Epoch:    30,  cost: 0.36544013
Epoch:    40,  cost: 0.21423372
Epoch:    50,  cost: 0.12625770


Evaluate Model Performance

In [10]:
def my_predict(m, X_test):
    ### START CODE HERE ###

    y_lin  = np.dot(X_test, m.wegt) + m.bias    # Linear Prediction
    y_pred = np.where(y_lin > 0, 1, -1)    # Predict Species. The result must be ? or ?. Use np.where()

    ### END CODE HERE ###
    return y_pred

from sklearn.metrics import accuracy_score

y_pred = my_predict(m, X_test)

accuracy_score(y_pred, y_test)

1.0

Support Vector Machine from scikit-learn

In [11]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')

# Training/Fitting the Model
svm.fit(X_train, y_train)

# Making Predictions
s_pred = svm.predict(X_test)

accuracy_score(s_pred, y_test)

1.0

### Test Model with a random sample


In [14]:
idx = np.random.randint(X_test.shape[0])
test_in = np.expand_dims(X_test[idx], axis=0)

species = ['setosa', 'versicolor']

y_pred = my_predict(m, test_in)
s_pred = svm.predict(test_in)

# Since prediction results are -1 or 1, (pred+1)//2 gives 0 or 1.
print('My prediction for Iris Species:', species[(y_pred[0]+1)//2])
print('SK prediction for Iris Species:', species[(s_pred[0]+1)//2])
print('Actual Iris Species:', species[(y_test[idx]+1)//2])

My prediction for Iris Species: setosa
SK prediction for Iris Species: setosa
Actual Iris Species: setosa
